In [1]:
from MyDataset import CifarDataModule

from art.project import ArtProject
from art.checks import CheckResultExists, CheckScoreExists, CheckScoreLessThan, CheckScoreGreaterThan
from art.steps import EvaluateBaseline, OverfitOneBatch, Overfit#, TransferLearning
from torchmetrics import Accuracy, Precision, Recall
import torch.nn as nn
from lightning.pytorch.callbacks import EarlyStopping

from MyDataset import CifarDataModule
from checks import CheckClassImagesExist, CheckLenClassNamesEqualToNumClasses
from steps import DataAnalysis
import math

In [2]:
%load_ext autoreload
%autoreload 2
from lightning import seed_everything
seed_everything(23)

Global seed set to 23


23

# Proper data analysis

In [3]:
project = ArtProject("Cifar100", CifarDataModule(batch_size=32))
project.add_step(DataAnalysis(), [
    CheckResultExists("number_of_classes"),
    CheckResultExists("class_names"),
    CheckResultExists("number_of_examples_in_each_class"),
    CheckResultExists("img_dimensions"),
    CheckClassImagesExist(),
    CheckLenClassNamesEqualToNumClasses()])
project.run_all()


Summary: 
Step: Data analysis, Model: , Passed: True. Results:

Code of the following steps was changed: Data analysis
 Rerun could be needed.


In [4]:
from torchmetrics import Accuracy
import torch.nn as nn
from art.steps import EvaluateBaseline

NUM_CLASSES = project.get_step(0).get_latest_run()["number_of_classes"]
accuracy_metric, ce_loss = Accuracy(task="multiclass", num_classes = NUM_CLASSES), nn.CrossEntropyLoss()
project.register_metrics([accuracy_metric, ce_loss])

In [5]:
from art.metrics import SkippedMetric
from models.baselines import MlBaseline, HeuristicBaseline, AlreadyExistingResNet20Baseline
from art.checks import CheckScoreExists
baselines = [HeuristicBaseline, MlBaseline, AlreadyExistingResNet20Baseline]
for baseline in baselines:
    project.add_step(
        step = EvaluateBaseline(baseline), 
        checks = [CheckScoreExists(metric=accuracy_metric)],
        skipped_metrics=[SkippedMetric(metric=ce_loss)]
    )


In [6]:
project.run_all()

Summary: 
Step: Data analysis, Model: , Passed: True. Results:

Step: Evaluate Baseline, Model: HeuristicBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.012299999594688416
Step: Evaluate Baseline, Model: MlBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.15649999678134918
Step: Evaluate Baseline, Model: AlreadyExistingResNet20Baseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.6883000135421753
Code of the following steps was changed: Data analysis
 Rerun could be needed.


In [7]:
from models.ResNet import ResNet18
from art.steps import CheckLossOnInit
from art.checks import CheckScoreCloseTo
from torch.cuda import is_available

EXPECTED_LOSS = -math.log(1/NUM_CLASSES)
print(EXPECTED_LOSS)

4.605170185988091


In [8]:
project.add_step(
        CheckLossOnInit(ResNet18),
        [CheckScoreCloseTo(metric=ce_loss,
                           value=EXPECTED_LOSS, rel_tol=0.1)]
    )

project.run_all()

Summary: 
Step: Data analysis, Model: , Passed: True. Results:

Step: Evaluate Baseline, Model: HeuristicBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.012299999594688416
Step: Evaluate Baseline, Model: MlBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.15649999678134918
Step: Evaluate Baseline, Model: AlreadyExistingResNet20Baseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.6883000135421753
Step: Check Loss On Init, Model: ResNet18, Passed: True. Results:
	MulticlassAccuracy-validate: 0.015399999916553497
	CrossEntropyLoss-validate: 5.087794780731201
Code of the following steps was changed: Data analysis, ResNet18_Check Loss On Init
 Rerun could be needed.


In [9]:
from art.steps import OverfitOneBatch
from art.checks import CheckScoreLessThan
project.add_step(OverfitOneBatch(ResNet18, number_of_steps=40),
            [CheckScoreLessThan(metric=ce_loss, value=0.05)])
project.run_all()

Summary: 
Step: Data analysis, Model: , Passed: True. Results:

Step: Evaluate Baseline, Model: HeuristicBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.012299999594688416
Step: Evaluate Baseline, Model: MlBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.15649999678134918
Step: Evaluate Baseline, Model: AlreadyExistingResNet20Baseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.6883000135421753
Step: Check Loss On Init, Model: ResNet18, Passed: True. Results:
	MulticlassAccuracy-validate: 0.015399999916553497
	CrossEntropyLoss-validate: 5.087794780731201
Step: Overfit One Batch, Model: ResNet18, Passed: True. Results:
	MulticlassAccuracy-train: 1.0
	CrossEntropyLoss-train: 0.00015361519763246179
Code of the following steps was changed: Data analysis, ResNet18_Check Loss On Init, ResNet18_Overfit One Batch
 Rerun could be needed.


In [ ]:
from art.steps import Overfit
from art.checks import CheckScoreGreaterThan

project.add_step(Overfit(ResNet18, max_epochs=10),
            [CheckScoreGreaterThan(metric=accuracy_metric, value=0.8)])

In [ ]:
project.run_all()

In [11]:
from art.steps import TransferLearning
from art.loggers import NeptuneLoggerAdapter

logger = NeptuneLoggerAdapter(project="mmaecki/transfer-learning", api_token="")
early_stopping = EarlyStopping('CrossEntropyLoss-validate', patience=6)
project.add_step(TransferLearning(ResNet18,
                                    freezed_trainer_kwargs={"max_epochs": 4,
                                                            "check_val_every_n_epoch": 2,
                                                            "callbacks": [early_stopping]},
                                    unfreezed_trainer_kwargs={"max_epochs": 50,
                                                            "check_val_every_n_epoch": 2,
                                                            "callbacks": [early_stopping]},
                                    keep_unfrozen=1,
                                    logger = logger,
                                    ),
                    [CheckScoreGreaterThan(metric=accuracy_metric, value=0.70)])

c:\Users\matri\anaconda3\envs\art\lib\site-packages\neptune\common\warnings.py:62: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  warnings.warn(


https://app.neptune.ai/mmaecki/transfer-learning/e/TRAN-5


In [12]:
project.run_all()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Using cache found in C:\Users\matri/.cache\torch\hub\facebookresearch_semi-supervised-ImageNet1K-models_master
c:\Users\matri\anaconda3\envs\art\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\matri\anaconda3\envs\art\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

  | Name  | Type             | Params
-------------------------------------------
0 | model | 

Sanity Checking: 0it [00:00, ?it/s]

c:\Users\matri\anaconda3\envs\art\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\matri\anaconda3\envs\art\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are us

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

c:\Users\matri\anaconda3\envs\art\lib\site-packages\lightning\pytorch\trainer\call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Using cache found in C:\Users\matri/.cache\torch\hub\facebookresearch_semi-supervised-ImageNet1K-models_master
c:\Users\matri\anaconda3\envs\art\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory c:\Users\matri\Polibuda\diploma\ART-Templates\{{cookiecutter.project_slug}}\.neptune\Untitled\TRAN-5\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 11.2 M
1 | loss  | CrossEntropy

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]